# Chapter 7: Measuring and Evaluating Returns

## Listing 7.1

In [1]:
from scipy.optimize import fsolve

def f(x):
    return 10+10/(1+x)-19.8/(1+x)**2
root = fsolve(f, 0)
print('The Dollar-Weighted Return is ', root.item())

The Dollar-Weighted Return is  -0.006681547693192119


## Listing 7.2

In [2]:
from datetime import datetime

cashflows = [10, 10, -19.8]
dates = ['2020-01-01', '2021-01-01', '2022-01-01']
dates = [datetime.strptime(dt, "%Y-%m-%d") for dt in dates]

def f(x, *args):
    cashflows, dates = args
    sumPVs = 0
    for i in range(len(cashflows)):
        Y = (dates[i] - dates[0]).days/365
        sumPVs += cashflows[i]/(1+x)**Y
    return sumPVs
root = fsolve(f, 0, args=(cashflows, dates))
print('The Dollar-Weighted Return is ', root.item())

The Dollar-Weighted Return is  -0.0066754853766419105


## Listing 7.3

In [3]:
import pandas_datareader as pdr
import yfinance as yf
import statsmodels.api as sm

start = '2005-02-01'
end = '2022-09-30'
rf_daily = pdr.DataReader('DGS1MO', 'fred', start=start, end=end)
ESG_prices = yf.download('SUSA', start=start, end=end)['Adj Close']
mkt_prices = yf.download('SPY', start=start, end=end)['Adj Close']
rf_monthly = rf_daily.resample('MS').first()
ESG_prices_monthly = ESG_prices.resample('MS').first()
mkt_prices_monthly = mkt_prices.resample('MS').first()
ESG_returns_monthly = ESG_prices_monthly.pct_change()
mkt_returns_monthly = mkt_prices_monthly.pct_change()
y = (ESG_returns_monthly - rf_monthly['DGS1MO']/100/12).dropna()
X = (mkt_returns_monthly - rf_monthly['DGS1MO']/100/12).dropna()
X = sm.add_constant(X) #A
model = sm.OLS(y,X)
results = model.fit()
print(results.summary()) 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.981
Model:                            OLS   Adj. R-squared:                  0.981
Method:                 Least Squares   F-statistic:                 1.088e+04
Date:                Fri, 29 Sep 2023   Prob (F-statistic):          3.27e-182
Time:                        11:20:07   Log-Likelihood:                 751.46
No. Observations:                 211   AIC:                            -1499.
Df Residuals:                     209   BIC:                            -1492.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      

## Multifactor models

In [1]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import getFamaFrenchFactors as gff

We saw that the one-factor alpha for stock $i$ is the constant $\alpha$ in the following regression:

$$
r_i - r_{f} = \alpha + \beta \, (r_{m} - r_{f}) + e_i
$$

Fama and French found that, historically from 1962-1989, small cap stocks outperformed large cap stocks and high book-to-market stocks ("value stocks") outperformed low book-to-market stocks ("growth stocks"). This potential irrationality in the market is sometimes referred to as an anomaly. This led to an extension of the one-factor model to a three-factor model:

$$
r - r_{f} = \alpha + b_i\, (r_{m} - r_{f}) + s_i \, \textrm{SMB} + h_i \, \textrm{HML} + e_i
$$

where the factor SMB (for small minus big) is a time series of returns of a portfolio that is long small cap stocks and short  large cap stocks, and HML (for high minus low) is a time series of returns for a portfolio that is long high book-to-market stocks and short low book-to-market stocks. Because the risk-free rate is subtracted from the market returns, it too can be considered a portfolio that is long the market and short a risk-free bond. In academia, the Fama-French alphas are computed for papers that claim they discovered an anomaly, to show that the anomaly is not merely a manifestation of one of these two anomalies from Fama and French. There is also a philosophical debate among academics about whether the outperformance of these factors is due to irrationality in the market, or whether they are proxies for exposures to systematic risks that are not captured by market risk. And some have argued that what seems like an irrationality in the market may not be an irrationality at all and may be the result of data mining. Indeed, growth stocks, which had significantly underperformed value stocks during the Fama-French period that ended in 1989, have significantly outperformed value stocks over the last 15 years.

Since Fama and French, many additional factors have been proposed. Another common factor is Momentum. There is a long body of literature, starting with Jagadeesh and Titman, that find that stocks that have outperformed over the last 12 months continue to outperform. At the same time, there was a debate in the academic literature about whether mutual funds have a "hot hand", which is the notion that mutual funds that have done well in the past are able to repeat their outperformance. Many academics, who were proponents of market efficiency, believed that any outperformance was due to luck and not repeatable. But there was some evidence that surprisingly found the opposite result. Mark Carhart argued that many winning funds had an exposure to momentum stocks, which is why they became winners. And if momentum stocks tend to outperform, then the hot hand is merely due to exposure to a Momentum factor. This led to the four factor Fama-French plus Carhart:

$$
r - r_{f} = \alpha + b_i\, (r_{m} - r_{f}) + s_i \, \textrm{SMB} + h_i \, \textrm{HML} + m_i \, \textrm{MOM} + e_i
$$

where MOM is a time series of returns of a portfolio of stocks that have had positive 12-month momentum and short stocks that have had negative momentum.

More recently, Fama and French have updated their model to include two additional factors, RMW, which is the difference in returns between a portfolio of companies that have high, or robust, operating profitability minus companies that have low, or weak, operating profitability, and CMA, which is the difference in returns between a portfolio of companies that have low, or conservative capital expenditures minus companies that have high, or aggressive, capital expenditures:

$$
r - r_{f} = \alpha + b_i\, (r_{m} - r_{f}) + s_i \, \textrm{SMB} + h_i \, \textrm{HML} + r_i \, \textrm{RMW} + c_i \, \textrm{CMA} + e_i
$$

To estimate the three-factor Fama-French alpha, we need to first download the time series of factor returns. The package `getFamaFrenchFactors` can be installed to help download those factors from Ken French's website, which maintains the data. After `pip install getFamaFrenchFactors`, downloading the factors is only a few lines of code:

In [25]:
import getFamaFrenchFactors as gff

ff3 = gff.famaFrench3Factor(frequency='m')
ff3.rename(columns={"date_ff_factors": 'Date'}, inplace=True)
ff3.set_index('Date', inplace=True)
ff3.head()

,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07-31,0.0296,-0.0256,-0.0243,0.0022
1926-08-31,0.0264,-0.0117,0.0382,0.0025
1926-09-30,0.0036,-0.0140,0.0013,0.0023
1926-10-31,-0.0324,-0.0009,0.0070,0.0032
1926-11-30,0.0253,-0.0010,-0.0051,0.0031


The data frequency options are either monthly or annually. Next we will download the daily prices for the ESG fund SUSA, and resample the data to monthly and compute returns.

In [26]:
ESG_prices = yf.download('SUSA', start='2005-02-01', end='2022-09-30')['Adj Close']
ESG_returns = ESG_prices.resample('M').last().pct_change().dropna()

[*********************100%***********************]  1 of 1 completed


In [27]:
ESG_returns.head()

Date
2005-03-31   -0.014168
2005-04-30   -0.029629
2005-05-31    0.055911
2005-06-30   -0.002435
2005-07-31    0.030833
Freq: M, Name: Adj Close, dtype: float64

Next, we will merge the Fama-French factor returns with the ETF returns

In [28]:
combined_returns = ff3.merge(ESG_returns,on='Date')
combined_returns.head()

,Mkt-RF,SMB,HML,RF,Adj Close
Date,,,,,
2005-03-31,-0.0197,-0.0140,0.0204,0.0021,-0.014168
2005-04-30,-0.0261,-0.0393,0.0006,0.0021,-0.029629
2005-05-31,0.0365,0.0289,-0.0064,0.0024,0.055911
2005-06-30,0.0057,0.0259,0.0282,0.0023,-0.002435
2005-07-31,0.0392,0.0290,-0.0078,0.0024,0.030833


Just as before with the one-factor alpha, we will regress the excess returns of the ETF on the returns of the factors, but now using the three factors.

In [29]:
X = combined_returns[['Mkt-RF', 'SMB', 'HML']]
y = combined_returns['Adj Close'] - combined_returns['RF']
X = sm.add_constant(X)
mod = sm.OLS(y, X).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.975
Model:                            OLS   Adj. R-squared:                  0.974
Method:                 Least Squares   F-statistic:                     2648.
Date:                Sun, 11 Dec 2022   Prob (F-statistic):          8.87e-165
Time:                        16:33:30   Log-Likelihood:                 745.57
No. Observations:                 211   AIC:                            -1483.
Df Residuals:                     207   BIC:                            -1470.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0004      0.000     -0.725      0.4

The beta coefficient on HML is negative but not significant, but the coefficient on SML is significantly negative, as expected since the ESG fund holds mostly larger cap stocks. However, the three-factor alpha is slighly negative, just like the one-factor alpha.

To get the three-factor plus Carhart alpha, simply download the factor returns four these four factors:

In [30]:
carhart4 = gff.carhart4Factor(frequency='m')
carhart4.head()

,date_ff_factors,Mkt-RF,SMB,HML,RF,MOM
0,1926-07-31,0.0296,-0.0256,-0.0243,0.0022,NaN
1,1926-08-31,0.0264,-0.0117,0.0382,0.0025,NaN
2,1926-09-30,0.0036,-0.0140,0.0013,0.0023,NaN
3,1926-10-31,-0.0324,-0.0009,0.0070,0.0032,NaN
4,1926-11-30,0.0253,-0.0010,-0.0051,0.0031,NaN


To get the five-factor alpha:

In [31]:
ff5 = gff.famaFrench5Factor(frequency='m')
ff5.head()

,date_ff_factors,Mkt-RF,SMB,HML,RMW,CMA,RF
0,1963-07-31,-0.0039,-0.0041,-0.0097,0.0068,-0.0118,0.0027
1,1963-08-31,0.0507,-0.0080,0.0180,0.0036,-0.0035,0.0025
2,1963-09-30,-0.0157,-0.0052,0.0013,-0.0071,0.0029,0.0027
3,1963-10-31,0.0253,-0.0139,-0.0010,0.0280,-0.0201,0.0029
4,1963-11-30,-0.0085,-0.0088,0.0175,-0.0051,0.0224,0.0027
